## MyVariant.info and MyGene.info Use Case

The following R script demonstrates the utility of MyVariant.info and MyGene.info R clients to annotate variants and prioritize candidate genes in patients with rare Mendelian diseases. This specific study uses data obtained from the database of phenotype and genotype (dbGaP) study...FASTQ files generated by Ng et al for the Miller syndrome study. FASTQ files were processed according to the Broad Institute’s best practices. Individual samples were aligned to the hg19 reference genome using BWA-MEM 0.7.10. Variants were called using GATK 3.3-0 HaplotypeCaller and quality scores were recalibrated using GATK VariantRecalibrator.

In [1]:
library(myvariant, quietly=TRUE)
library(mygene, quietly=TRUE)
library(VariantAnnotation, quietly=TRUE)
library(GO.db, quietly=TRUE)
source("https://raw.githubusercontent.com/Network-of-BioThings/myvariant.info/master/docs/ipynb/mendelian.R")
setwd("~/sulab/myvariant/vcf/recal")
vcf.files <- paste(getwd(), list.files(getwd()), sep="/")


Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Creating a generic function for ‘nchar’ from package ‘base’ in package ‘S4Vectors’

Attaching package: ‘VariantAnnotation’

The following object is masked from ‘package:base’:

    tabulate

Welcome to Bioconductor

    Vignettes contain intr

mendelian.R defines some helper functions that are used in the analysis downstream of retrieving annotations:

`replaceWith0` - replaces all NAs in a data.frame with 0.

`rankByCaddScore` - for prioritizing genes by deleteriousness (scaled CADD score).

---
## Annotating variants with MyVariant.info
The following function reads in each output VCF file using the VariantAnnotation package available from Bioconductor. Install with `biocLite("VariantAnnotation")`. `formatHgvs` (from the ***myvariant*** Bioconductor package) is a function that reads the genomic location and variant information from the VCF to create HGVS IDs which serve as a primary key for each variant. The function `getVariants` makes the queries to MyVariant.info to retrieve annotations.

In [2]:
getVars <- function(vcf.file){
  cat(paste("Processing ", vcf.file, "...\n", sep=" "))
  vcf <- readVcf(vcf.file, genome="hg19")
  vcf <- vcf[isSNV(vcf)]
  vars <- rowRanges(vcf)
  vars <- as(vars, "DataFrame")
  vars$query <- formatHgvs(vcf, "snp")
  annotations <- getVariants(vars$query, fields=c("dbnsfp.genename", "dbnsfp.1000gp1.af", 
                                                  "exac.af", "cadd.consequence", "cadd.phred"), verbose=FALSE)
  annotations[c('DP', 'FS', 'QD')] <- info(vcf)[c('DP', 'FS', 'QD')]
  annotations <- replaceWith0(annotations)
  annotations
}

vars <- lapply(vcf.files, getVars)

Processing  /Users/cyrusafrasiabi/recal/subject01_recalibrate_SNP_vqsr.vcf ...
found header lines for 3 ‘fixed’ fields: ALT, QUAL, FILTER 
found header lines for 24 ‘info’ fields: AC, AF, ..., VQSLOD, culprit 
found header lines for 5 ‘geno’ fields: GT, AD, DP, GQ, PL 


Concatenating data, please be patient.


Processing  /Users/cyrusafrasiabi/recal/subject02_recalibrate_SNP_vqsr.vcf ...
found header lines for 3 ‘fixed’ fields: ALT, QUAL, FILTER 
found header lines for 24 ‘info’ fields: AC, AF, ..., VQSLOD, culprit 
found header lines for 5 ‘geno’ fields: GT, AD, DP, GQ, PL 


Concatenating data, please be patient.


Processing  /Users/cyrusafrasiabi/recal/subject03_recalibrate_SNP_vqsr.vcf ...
found header lines for 3 ‘fixed’ fields: ALT, QUAL, FILTER 
found header lines for 24 ‘info’ fields: AC, AF, ..., VQSLOD, culprit 
found header lines for 5 ‘geno’ fields: GT, AD, DP, GQ, PL 


Concatenating data, please be patient.


Processing  /Users/cyrusafrasiabi/recal/subject04_recalibrate_SNP_vqsr.vcf ...
found header lines for 3 ‘fixed’ fields: ALT, QUAL, FILTER 
found header lines for 24 ‘info’ fields: AC, AF, ..., VQSLOD, culprit 
found header lines for 5 ‘geno’ fields: GT, AD, DP, GQ, PL 


Concatenating data, please be patient.


We then filter each of the VCF files based on annotations.

In [3]:
countNumGenesMutatedInAllPatients <- function(inp, text) {
    cat(paste(nrow(subset(data.frame(table(unlist(lapply(inp, function(i) unique(i$dbnsfp.genename))))), 
    Freq == 4)), text))
}
                                                         
countNumGenesMutatedInAllPatients(vars, "common genes are mutated amongst patients")                                                         

2442 common genes are mutated amongst patients

---
## Filtering Steps
The next steps apply annotation based filters by subsetting each data.frame by those variants that meet each criteria.

1) The first filter utilizes annotations output by HaplotypeCaller from GATK. Here we keep variants with at least 8 reads, Fisher Strand bias less than 30, and quality over depth greater than 2. 

In [4]:
filter1 <- lapply(vars, function(i) subset(i, DP > 8 & FS < 30 & QD > 2))

countNumGenesMutatedInAllPatients(filter1, "genes remain after filtering for coverage and strand bias") 

2309 genes remain after filtering for coverage and strand bias

2) Mendelian diseases are most likely to be caused by nonsynonymous mutations. We can take advantage of the CADD database that denotes the mutation type in the field "cadd.consequence".

In [5]:
filter2 <- lapply(filter1, function(i) subset(i, cadd.consequence %in% c("NON_SYNONYMOUS", "STOP_GAINED", "STOP_LOST", 
                                           "CANONICAL_SPLICE", "SPLICE_SITE")))

countNumGenesMutatedInAllPatients(filter2, "genes remain after filtering for nonsynonymous and splice site variants")

1918 genes remain after filtering for nonsynonymous and splice site variants

3) The third filter keeps rare variants according to the ExAC data set with allele frequency < 0.01. Rare diseases are likely caused by mutations that have not been documented yet.

In [6]:
filter3 <- lapply(filter2, function(i) subset(i, exac.af < 0.01))

countNumGenesMutatedInAllPatients(filter3, "genes remain after filtering for allele frequency < 0.01 in the ExAC dataset")

19 genes remain after filtering for allele frequency < 0.01 in the ExAC dataset

4) The last filter keeps rare variants according to the 1000 Genomes Project with allele frequency < 0.01.

In [7]:
filter4 <- lapply(filter3, function(i) subset(i, sapply(dbnsfp.1000gp1.af, function(j) j < 0.01 )))
    
top.genes <- subset(data.frame(table(unlist(lapply(filter4, function(i) unique(i$dbnsfp.genename))))), Freq == 4)
top.genes <- subset(top.genes, !(Var1 %in% c("NULL", 0)))
cat(paste(nrow(top.genes), "genes remain after filtering for allele frequency < 0.01 in 1000 Genomes Project"))

9 genes remain after filtering for allele frequency < 0.01 in 1000 Genomes Project

In [8]:
top.genes$Var1

[1] CDC27   CDON    CTBP2   DHODH   FAM104B HYDIN   KIR3DL3 PIK3R3  XKR3   
968 Levels: 0 ABCA2 ABCA6 ABCB6 ABCC10 ABCC11 ABCC5 ABHD5 ABLIM3 ABTB1 ... ZWINT

---
## Prioritizing genes
We can prioritize the genes that contain variants in each of the patients according to CADD (deleteriousness) score. `rankByCaddScore` extracts the average CADD scores of the variants in each gene per patient and ranks in descending order.

In [9]:
ranked <- rankByCaddScore(top.genes$Var1, filter4)
ranked

,gene,cadd.phred
1,DHODH,26.81
2,CTBP2,21.385
3,PIK3R3,20.7
4,HYDIN,18.91
5,CDC27,18.545
6,XKR3,15.72
7,CDON,10.02
8,FAM104B,7.2055
9,KIR3DL3,1.012


---
##Annotating candidate genes with MyGene.info
Now we can annotate the genes using the `queryMany` function from ***mygene*** which can be install from Bioconductor (`biocLite("mygene")`). We specify `scopes=symbol` and request annotations by the `field` parameter. 

In [10]:
res <- data.frame(queryMany(ranked$gene, scopes="symbol", species="human", fields=c("go.BP", "name", "MIM", "uniprot")))

Finished


In [11]:
res$go.BP

[[1]]
   evidence                                                   term         id
1       TAS                pyrimidine nucleobase metabolic process GO:0006206
2       IBA   'de novo' pyrimidine nucleobase biosynthetic process GO:0006207
3       IEA                                       female pregnancy GO:0007565
4       IEA                                              lactation GO:0007595
5       IEA                                   response to caffeine GO:0031000
6       IEA                                       response to drug GO:0042493
7       IEA                                 response to starvation GO:0042594
8       IEA               positive regulation of apoptotic process GO:0043065
9       IEA                     'de novo' UMP biosynthetic process GO:0044205
10      TAS                       small molecule metabolic process GO:0044281
11      TAS             pyrimidine nucleoside biosynthetic process GO:0046134
12      TAS nucleobase-containing small molecule metabolic

---
# GO annotation
We can then use the Bioconductor package ***GO.db*** to map GO IDs to GO:0008152, the biological process id for metabolic process.

In [12]:
miller.bp <- lapply(res$go.BP, function(i) unlist(i$id))
bp.ancestor <- lapply(miller.bp, function(i) sapply(i, function(j) "GO:0008152" %in% unlist(GOBPANCESTOR[[j]])))
candidate.genes <- ranked$gene[sapply(bp.ancestor, function(i) TRUE %in% i)]
candidate.genes

[1] DHODH  CTBP2  PIK3R3 CDC27  CDON  
Levels: CDC27 CDON CTBP2 DHODH FAM104B HYDIN KIR3DL3 PIK3R3 XKR3

The final candidate gene list contains only genes involved in metabolic processes where perterbation is more likely to result in mendelian disease. Through the use of MyVariant.info and MyGene.info annotation services, we have narrowed the candidate gene list from 2589 genes to seven. While we cannot fully remove the process of biological interpretation from an analysis, we have greatly reduced the burden.